In [4]:
import veryfi
import os
import json
import re
import pandas as pd

# Configuración de credenciales de Veryfi
client_id = 'vrfhjAs5kPIee2Yvy7F0osVV5MX6zXKFTriJyLS'
client_secret = 'NJoXZcE25MBxv97merROYW5CvYBJLmNwrGDoYEWzv970T0fyeE2pxAXlP93huX46Bf1NbgM6jGzVb3LzSrTWZYDEGcyaruHwfWAr75EfA87rcoo29NLYzWRWJdogbc1N'
username = 'juanpamon.13'
api_key = 'f604ee697fd73c0ba1bb99a536bcde0b'

client = veryfi.Client(client_id, client_secret, username, api_key)

# Directorios
input_folder = "Documents"
veryfi_output_folder = "veryfi_jsons"
processed_output_folder = "results"

os.makedirs(veryfi_output_folder, exist_ok=True)
os.makedirs(processed_output_folder, exist_ok=True)

def clean_ocr_text(text):
    """
    Limpia el OCR eliminando saltos de página y encabezados/pies de página.
    Por ejemplo, elimina líneas que contengan:
      - "Invoice" solas
      - "Page X of Y"
    """
    # Reemplaza saltos de página por saltos de línea
    text = text.replace("\f", "\n")
    if not re.search(r'Description\s+Quantity\s+Rate\s+Amount', text, flags=re.IGNORECASE):
        print("El documento no cumple con el formato de invoice esperado; se omitirá este documento.")
        return None
    # Elimina líneas que sean encabezados de página o pies (puedes ajustar los patrones)
    text = re.sub(r'^\s*(Invoice|Page\s+\d+\s+of\s+\d+).*$', '', text, flags=re.MULTILINE)
    # Elimina líneas vacías extra
    text = "\n".join([line for line in text.splitlines() if line.strip() != ""])
    return text

def is_valid_row(description):
    """
    Devuelve False si la descripción corresponde a un encabezado o
    contenido no deseado (por cambio de página u otro) que se ha
    incluido por error en la sección de ítems.
    """
    desc = description.strip().lower()
    if desc.startswith("po box"):
        return False
    if "invoice date" in desc:
        return False
    if desc.startswith("micro merchant systems"):
        return False
    if desc.startswith("account no."):
        return False
    if desc == "description":
        return False
    if desc.startswith("o-") and len(desc) < 15:
        return False
    # Filtra información de vendor que aparece en cambio de página
    if "incentx" in desc and "main st" in desc:
        return False
    # Filtra códigos tipo "w-300387"
    if re.fullmatch(r'w-\d+', desc):
        return False
    return True

def clean_description(desc):
    """
    Elimina de la descripción fragmentos indeseados (por ejemplo,
    partes que provengan de encabezados de página).
    En este ejemplo se elimina cualquier texto a partir de "Invoice switch".
    Ajusta el patrón según los casos que observes.
    """
    cleaned = re.sub(r'Invoice\s+switch.*', '', desc, flags=re.IGNORECASE)
    return cleaned.strip()

def extract_invoice_data(ocr_text):
    """
    Extrae la información clave de la factura desde el texto OCR.
    """
    # Primero, limpiamos el OCR para eliminar encabezados/pies de página
    
    
    invoice_data = {}

    # Vendor Name
    vendor_match = re.search(r'Please make payments to:\s*(.+)', ocr_text)
    invoice_data["vendor_name"] = vendor_match.group(1).strip() if vendor_match else "Unknown"

    # Vendor Address (fijo)
    vendor_address_match = re.search(r"\n([\w\s,]+ \d{5}(?:-\d{4})?)\nPO Box", ocr_text)
    if vendor_address_match:
        address = vendor_address_match.group(1).strip()
        address_parts = address.split()[-3:]  # Ejemplo: ciudad, estado y código postal
        invoice_data["vendor_address"] = " ".join(address_parts)
    else:
        invoice_data["vendor_address"] = "Unknown"

    # Invoice Number y Bill To Name (ajustado al nuevo formato)
    invoice_match = re.search(
        r'Invoice Date Due Date\tInvoice No\.\n\t(\d{2}/\d{2}/\d{2})\t(\d{2}/\d{2}/\d{2})\t([\d]+)\n\n([^\n]+)',
        ocr_text
    )
    if invoice_match:
        invoice_data["date"] = invoice_match.group(1)           # Ejemplo: "01/12/24"
        invoice_data["invoice_number"] = invoice_match.group(3)   # Ejemplo: "9230090"
        invoice_data["bill_to_name"] = invoice_match.group(4).strip()  # Ejemplo: "IncentX"
    else:
        invoice_data["date"] = "Unknown"
        invoice_data["invoice_number"] = "Unknown"
        invoice_data["bill_to_name"] = "Unknown"

    ocr_text = clean_ocr_text(ocr_text)
    if ocr_text is None:
        # Si no se detecta el formato esperado, se omite este documento.
        raise ValueError("El documento no cumple con el formato de invoice esperado.")
    
    

    # --- Paso 1: Extraer la sección de precios ---
    # Usamos patrones con flag MULTILINE para encontrar el encabezado y la línea de totales.
    start_pattern = r"^\s*Description\s+Quantity\s+Rate\s+Amount\s*$"
    end_pattern = r"^\tTotal\s+USD"

    match_start = re.search(start_pattern, ocr_text, flags=re.MULTILINE)
    match_end = re.search(end_pattern, ocr_text, flags=re.MULTILINE)
    
    if match_start and match_end:
        price_section = ocr_text[match_start.end():match_end.start()]
    else:
        raise ValueError("No se pudo identificar la sección de lista de precios.")

    oct_text = price_section

    # --- Paso 2: Procesar oct_text para reconstruir la tabla de 4 columnas ---
    rows = []
    current_row = None
    for line in oct_text.splitlines():
        if "\t" in line:
            tokens = line.split("\t")
            # Si hay menos de 4 tokens, se asume que la línea es parte de la descripción.
            if len(tokens) < 4:
                if current_row is not None:
                    current_row[0] += " " + line.strip()
                continue
            if current_row is not None:
                rows.append(current_row)
            description_tokens = tokens[:-3]
            description = " ".join(tok.strip() for tok in description_tokens if tok.strip())
            col1 = tokens[-3].strip()  # Cantidad
            col2 = tokens[-2].strip()  # Precio
            col3 = tokens[-1].strip()  # Total
            current_row = [description, col1, col2, col3]
        else:
            if current_row is not None:
                current_row[0] += " " + line.strip()
    if current_row is not None:
        rows.append(current_row)

    df = pd.DataFrame(rows, columns=["Descripción", "Col1", "Col2", "Col3"])

    # --- Paso 3: Extraer los ítems (filtrando falsos positivos) ---
    items = []
    for index, row in df.iterrows():
        desc = row["Descripción"]
        if not is_valid_row(desc):
            continue
        desc = clean_description(desc)
        try:
            quantity = float(row["Col1"].replace(',', ''))
        except (AttributeError, ValueError):
            quantity = 0.0
        try:
            price = float(row["Col2"].replace(',', ''))
        except (AttributeError, ValueError):
            price = 0.0
        try:
            total = float(row["Col3"].replace(',', '').replace('$', ''))
        except (AttributeError, ValueError):
            total = 0.0
        items.append({
            "sku": "Unknown",
            "description": desc,
            "quantity": quantity,
            "tax_rate": None,
            "price": price,
            "total": total
        })
    invoice_data["items"] = items
    return invoice_data

# Procesar todos los archivos PDF en la carpeta Documents
for file_name in os.listdir(input_folder):
    if file_name.endswith(".pdf"):
        file_path = os.path.join(input_folder, file_name)
        print(f"Procesando: {file_name}")
        
        json_result = client.process_document(file_path)
        
        veryfi_output_file = os.path.join(veryfi_output_folder, f"{file_name}.json")
        with open(veryfi_output_file, "w", encoding="utf-8") as f:
            json.dump(json_result, f, ensure_ascii=False, indent=4)
        
        print(f"Resultado guardado en: {veryfi_output_file}")
        
        with open(veryfi_output_file, "r", encoding="utf-8") as f:
            data = json.load(f)
            ocr_text = data.get("ocr_text", "")
            try:
                invoice_info = extract_invoice_data(ocr_text)
            except ValueError as ve:
                print(f"El documento '{file_name}' se omitirá: {ve}")
                continue
        
        processed_output_file = os.path.join(processed_output_folder, f"{file_name}.json")
        with open(processed_output_file, "w", encoding="utf-8") as f:
            json.dump(invoice_info, f, ensure_ascii=False, indent=4)
        
        print(f"Datos procesados guardados en: {processed_output_file}")

print("Procesamiento completado.")

Procesando: CertificadoPos_1000193765.pdf
Resultado guardado en: veryfi_jsons\CertificadoPos_1000193765.pdf.json
El documento no cumple con el formato de invoice esperado; se omitirá este documento.
El documento 'CertificadoPos_1000193765.pdf' se omitirá: El documento no cumple con el formato de invoice esperado.
Procesando: synth-switch_v5-14.pdf
Resultado guardado en: veryfi_jsons\synth-switch_v5-14.pdf.json
Datos procesados guardados en: results\synth-switch_v5-14.pdf.json
Procesando: synth-switch_v5-4.pdf
Resultado guardado en: veryfi_jsons\synth-switch_v5-4.pdf.json
Datos procesados guardados en: results\synth-switch_v5-4.pdf.json
Procesando: synth-switch_v5-68.pdf
Resultado guardado en: veryfi_jsons\synth-switch_v5-68.pdf.json
Datos procesados guardados en: results\synth-switch_v5-68.pdf.json
Procesando: synth-switch_v5-7.pdf
Resultado guardado en: veryfi_jsons\synth-switch_v5-7.pdf.json
Datos procesados guardados en: results\synth-switch_v5-7.pdf.json
Procesando: synth-switch_v5